In [19]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # 指定使用 GPU 3
import torch

In [21]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
import keras

import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
%env TOKENIZERS_PARALLELISM=false

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1
env: TOKENIZERS_PARALLELISM=false


In [22]:
import pandas as pd 
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding
)
from datasets import Dataset
from glob import glob
import gc
import torch
from scipy.special import softmax

In [23]:
import spacy
nlp = spacy.load("en_core_web_sm")
with open('kaggle/input/english-word-hx/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)

## 数据load和处理

In [24]:

keras.utils.set_random_seed(42) ##每次的随机值相同，方便复现
keras.mixed_precision.set_global_policy("mixed_float16")



In [25]:
import polars as pl
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
TEST_DATA_PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
MAX_LENGTH = 1024
EVAL_BATCH_SIZE = 1


## 添加模型

In [26]:
# Load model directlyfrom transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("addy88/argument-classifier")
model = AutoModelForSequenceClassification.from_pretrained("addy88/argument-classifier")

In [27]:
PATH = '/home/mcq2/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'

train = pl.read_csv(PATH + "train.csv").with_columns(columns) 
test = pl.read_csv(PATH + "test.csv").with_columns(columns)

### 小型预处理


In [28]:
def cut(tmp):   
    tmp = tmp.with_columns(pl.col('full_text').str.slice(0, 513).alias
    ("cut"))
    return tmp

test = cut(test)

df =pd.DataFrame({ "cut": test['cut']})

In [29]:
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none"
    
)
def tokenize(sample):
    return tokenizer(sample['cut'], max_length=MAX_LENGTH, truncation=True)

trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=DataCollatorWithPadding(tokenizer), 
        tokenizer=tokenizer
    )


In [15]:
ds = Dataset.from_pandas(df).map(tokenize).remove_columns(['cut'])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [30]:
preds = trainer.predict(ds).predictions

In [31]:
predictions = []
predictions.append(softmax(preds, axis=-1))

In [32]:
print(predictions)

[array([[0.184659  , 0.815341  ],
       [0.86983293, 0.13016711],
       [0.12448   , 0.87552005]], dtype=float32)]


In [ ]:
deberta_oof = joblib.load('/kaggle/input/aes2-400-20240419134941/oof.pkl')
print(deberta_oof.shape, train_feats.shape)

for i in range(6):
    train_feats[f'deberta_oof_{i}'] = deberta_oof[:, i]

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ', len(feature_names)) 

In [ ]:
class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

In [ ]:
def tokenize(sample):
    return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

df_test = pd.read_csv(TEST_DATA_PATH)
ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=EVAL_BATCH_SIZE, 
    report_to="none"
)

predictions = []
for model in models:
    model = AutoModelForSequenceClassification.from_pretrained(model)
    trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=DataCollatorWithPadding(tokenizer), 
        tokenizer=tokenizer
    )
    
    preds = trainer.tokenizer.predict(ds).predictions
    predictions.append(softmax(preds, axis=-1))  #predictions.append(...) 将转换后的概率分布添加到 predictions 列表中，用于进一步处理或评估。
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    
# predicted_score = 0.

# for p in predictions:
#     predicted_score += p
    
# predicted_score /= len(predictions)

In [ ]:
preds = trainer.predict(ds).predictions

def predict_chunk(test: pd.DataFrame) -> pd.DataFrame:

        submission = pd.DataFrame(np.ones((len(test), 2)))
        submission.columns = ['text_id', 'score_cut']
        submission["text_id"] = test["text_id"]
        # print(test["text_id"].head(5))
        # print(submission["text_id"].head(5))
        # sort by length to speed up inference
        test['tokenize_length'] = [len(tokenizer(text)['input_ids']) for text in test['full_text'].values]
        test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)

        # test_dataset = TestDataset(CFG, test)
        # test_loader = DataLoader(test_dataset,
        #                          batch_size=CFG.batch_size,
        #                          shuffle=False,
        #                          collate_fn=DataCollatorWithPadding(tokenizer=CFG.tokenizer, padding='longest'),
        #                          num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
        predictions = []

        
        #model.load_state_dict(state['model'], strict=False)
        for i in 
        prediction = trainer.predict(ds).predictions

        predictions.append(prediction)
        # del model, state, prediction; 
        # gc.collect()
        # torch.cuda.empty_cache()

        #predictions = np.mean(predictions, axis=0)

        ## 把prediction 放在test的一个列里
        test['score_cut'] = predictions


        submission = submission.drop(['score_cut']).merge(test[['text_id'] + ['score_cut']], on='text_id', how='left')
#         display(submission.head())
        submission[['text_id'] + ['score_cut']].to_csv(f'submission.csv', index=False)
        #torch.cuda.empty_cache()
        #gc.collect()
        return submission


if __name__ == "__main__":
    test = pd.read_csv(
            '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv'
    ).rename(columns={"essay_id": "text_id"})
    submission = predict(test)
    submission.to_csv(f'submission.csv', index=False)


In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

# Train and evaluate
trainer.train()
trainer.evaluate()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-model")


model = lgb.LGBMRegressor(
                    objective = qwk_obj,
                    metrics = 'None',
                    learning_rate = 0.05,
                    max_depth = 5,
                    num_leaves = 10,
                    colsample_bytree=0.3,
                    reg_alpha = 0.7,
                    reg_lambda = 0.1,
                    n_estimators=700,
                    random_state=412,
                    extra_trees=True,
                    class_weight='balanced',
                    verbosity = - 1)

        predictor = model.fit(X_train_fold,
                              y_train_fold,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
                              eval_metric=quadratic_weighted_kappa,
                              callbacks=callbacks)
        models.append(predictor)
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

In [ ]:
import pandas as pd
import fb3_deberta_family_inference_9_28_updated

if len(df_test) < 10:
    fb3_predicted = fb3_deberta_family_inference_9_28_updated.predict_chunk(
        df_test.rename(columns={"essay_id": "text_id"})
    )
else:
    fb3_predicted = fb3_deberta_family_inference_9_28_updated.predict(
        df_test.rename(columns={"essay_id": "text_id"})